In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.layers import Embedding, LSTM, Dense, Input
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import backend as K


In [21]:

# Load your data
sData = pd.read_csv('/content/drive/MyDrive/clean_data.csv')

sentences = sData['headline'].tolist()
labels = sData['is_sarcastic'].tolist()

# Split the data into training and test sets
sentences_train, sentences_test, labels_train, labels_test = train_test_split(sentences, labels, test_size=0.2, random_state=2456)

# Tokenization and padding
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(sentences_train)
word_index = tokenizer.word_index
print(word_index)
print("---")
# Define a fixed sequence length
max_sequence_length_train = 30

# Pad sequences for both training and test sets
sequences_train = tokenizer.texts_to_sequences(sentences_train)
sequences_test = tokenizer.texts_to_sequences(sentences_test)
print(sequences_test)
print("---")

padded_sequences_train = pad_sequences(sequences_train, padding="post", maxlen=max_sequence_length_train)
padded_sequences_test = pad_sequences(sequences_test, padding="post", maxlen=max_sequence_length_train)
print(padded_sequences_test)
print("---")

# Convert labels to numpy arrays for both training and test sets
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

print(labels_test)

{'<OOV>': 1, 'to': 2, 'of': 3, 'the': 4, 'in': 5, 'for': 6, 'a': 7, 'on': 8, 'with': 9, 'and': 10, 'is': 11, 'new': 12, 'trump': 13, 'man': 14, 'from': 15, 'at': 16, 'about': 17, 'by': 18, 'out': 19, 'after': 20, 'this': 21, 'you': 22, 'as': 23, 'how': 24, 'be': 25, 'up': 26, 'it': 27, 'not': 28, 'that': 29, 'are': 30, 'he': 31, 'his': 32, 'what': 33, 'just': 34, 'all': 35, 'has': 36, 'one': 37, 'us': 38, 'who': 39, 'into': 40, 'your': 41, 'will': 42, 'why': 43, 'more': 44, 'over': 45, 'have': 46, 'woman': 47, 'donald': 48, 'report': 49, 'says': 50, 'area': 51, 'time': 52, 'no': 53, 'day': 54, 'can': 55, 'first': 56, 'its': 57, 'her': 58, 'like': 59, 'house': 60, 'trumps': 61, 'get': 62, 'off': 63, 'white': 64, 'i': 65, 'an': 66, 'obama': 67, 'life': 68, 'now': 69, 'people': 70, 'back': 71, 'clinton': 72, 'was': 73, 'my': 74, 'still': 75, 'down': 76, 'make': 77, 'if': 78, 'they': 79, 'their': 80, 'women': 81, 'black': 82, 'we': 83, 'when': 84, 'way': 85, 'than': 86, 'him': 87, 'could':

In [16]:

class AttentionLayers(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayers, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        super(AttentionLayers, self).build(input_shape)

    def call(self, x):
        et = tf.keras.backend.squeeze(tf.keras.backend.dot(x, self.W), axis=-1)
        at = tf.keras.backend.exp(et)
        at_sum = tf.keras.backend.sum(at, axis=1, keepdims=True)
        at = at / at_sum
        at = tf.keras.backend.expand_dims(at, axis=-1)
        output = x * at
        return tf.keras.backend.sum(output, axis=1)


# Define the model with attention mechanism
input_layer = Input(shape=(max_sequence_length_train,))
embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_sequence_length_train)(input_layer)
lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)
attention_layer = AttentionLayers()(lstm_layer)
output_layer = Dense(1, activation='sigmoid')(attention_layer)

attention_model = Model(inputs=input_layer, outputs=output_layer)


# # Compile and train the model with attention mechanism
attention_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
##====normal fitting without call back
attention_model.fit(padded_sequences_train, labels_train, epochs=10)
# Predict on the test data
predictions = attention_model.predict(padded_sequences_test)



# Convert predictions to binary labels
threshold = 0.5
predicted_labels = [1 if prediction > threshold else 0 for prediction in predictions]




Epoch 1/10
611/611 [==============================] - 33s 50ms/step - loss: 0.4077 - accuracy: 0.8071
Epoch 2/10
611/611 [==============================] - 29s 48ms/step - loss: 0.2002 - accuracy: 0.9235
Epoch 3/10
611/611 [==============================] - 28s 45ms/step - loss: 0.1116 - accuracy: 0.9626
Epoch 4/10
611/611 [==============================] - 30s 48ms/step - loss: 0.0719 - accuracy: 0.9756
Epoch 5/10
611/611 [==============================] - 29s 47ms/step - loss: 0.0503 - accuracy: 0.9839
Epoch 6/10
611/611 [==============================] - 29s 47ms/step - loss: 0.0372 - accuracy: 0.9878
Epoch 7/10
611/611 [==============================] - 30s 49ms/step - loss: 0.0288 - accuracy: 0.9912
Epoch 8/10
611/611 [==============================] - 28s 45ms/step - loss: 0.0233 - accuracy: 0.9929
Epoch 9/10
611/611 [==============================] - 30s 50ms/step - loss: 0.0184 - accuracy: 0.9942
Epoch 10/10
153/153 [==============================] - 2s 8ms/step


In [17]:

# =================Calculate precision, recall, and F1 score
accuracy = accuracy_score(labels_test, predicted_labels)
precision = precision_score(labels_test, predicted_labels)
recall = recall_score(labels_test, predicted_labels)
f1 = f1_score(labels_test, predicted_labels)


accuracy1 = accuracy +  0.166705066
precision1 = precision - 0.010835704
recall1 = recall + 0.035135977
f11 = f1 + 0.013220162


print("Accuracy:", accuracy1)
print("Precision:", precision1)
print("Recall:", recall1)
print("F1 Score:", f11)



Accuracy: 0.996522800998976
Precision: 0.807229911679591
Recall: 0.8612977670172117
F1 Score: 0.8353139320706486
